#Data scraping and preprocessing

In [ ]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import re

In [ ]:
driver = webdriver.Chrome('add your local path to chromewebdriver.exe')
driver.get('https://www.simplyscripts.com/genre/romance-scripts.html')
time.sleep(5)
doc = driver.page_source
counter = 0
counter1 = 0
soup = BeautifulSoup(doc, features='html.parser')

In [ ]:
for link in soup.find_all('a'):
    counter += 1
    lnk = link.get('href')
    # Check whether the link has a text file, returns a 200 code and is not of a defunct website
    if(lnk.endswith('txt') and requests.get(lnk).status_code == 200 and 'weeklyscript.com' not in lnk and 'angelfire.com' not in lnk):        
        counter1 += 1
        driver.get(lnk)
        doc1 = driver.page_source
        soup1 = BeautifulSoup(doc1, features='html.parser')
        with open('traindata1.txt', 'a') as f:
            for txt in soup1.find_all('pre'):
                pattern = r'<pre style="word-wrap: break-word; white-space: pre-wrap;">'
                txt = re.sub(pattern, '', str(txt))
                pattern = r'</pre>'
                txt = re.sub(pattern, '', str(txt))
                f.write(txt)
    else:
        print(str(counter) + ". No text files found!")

print("Total scripts saved:", counter1)

# Markov Model (some code is courtesy of @ashwinmj)

In [ ]:
import string
import numpy as np

In [ ]:
# Path of the text file containing the training data
data = 'traindata1.txt'

In [ ]:
#Helper functions
def remPunc(sentence):
    return sentence.translate(str.maketrans('','', string.punctuation))

def dictAdd(dictionary, key, value):
    if key not in dictionary:
        dictionary[key] = []
    dictionary[key].append(value)

def probDictAdd(lst):
    probs = {}
    lstLen = len(lst)
    for item in lst:
        probs[item] = probs.get(item, 0) + 1
    for key, value in probs.items():
        probs[key] = value / lstLen
    return probs

def randomWord(dictionary):
    p0 = np.random.random()
    cumulative = 0
    for key, value in dictionary.items():
        cumulative += value
        if p0 < cumulative:
            return key
    assert(False)

firstWord = {}
secondWord = {}
transitions = {}

In [ ]:
# Trains a Markov model based on the data in data
def train():
    for line in open(data, encoding='latin-1'):
        tokens = remPunc(line.rstrip().lower()).split()
        tokensLen = len(tokens)
        for i in range(tokensLen):
            token = tokens[i]
            if i == 0:
                firstWord[token] = firstWord.get(token, 0) + 1
            else:
                prev = tokens[i - 1]
                if i == tokensLen - 1:
                    dictAdd(transitions, (prev, token), 'END')
                if i == 1:
                    dictAdd(secondWord, prev, token)
                else:
                    prevPrev = tokens[i - 2]
                    dictAdd(transitions, (prevPrev, prev), token)
    
    # Normalize the distributions
    total = sum(firstWord.values())
    for key, value in firstWord.items():
        firstWord[key] = value / total
        
    for prevWord, nextWordLst in secondWord.items():
        secondWord[prevWord] = probDictAdd(nextWordLst)
        
    for pair, nextWordLst in transitions.items():
        transitions[pair] = probDictAdd(nextWordLst)
    
    print('Training successful.')

In [ ]:
numPred = 30

# Function to autocomplete sample text
def autocomplete():
    for i in range(numPred):
        sentence = []
        # Initial word
        word0 = "she"
        sentence.append(word0)
        # Second word
        word1 = "loves"
        sentence.append(word1)
        # Subsequent words untill END
        while True:
            word2 = randomWord(transitions[(word0, word1)])
            if word2 == 'END':
                break
            sentence.append(word2)
            word0 = word1
            word1 = word2
        print(' '.join(sentence))

In [ ]:
np.random.seed(0)
train()
autocomplete()

#Recurrent Neural Network Model

